# imports

In [1]:
from utils import *
from model_utils import *
from data import *
from c2d_models import *

import torch
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries

# https://github.com/marcoancona/DeepExplain
from deepexplain.tensorflow import DeepExplain
from lime import lime_image

# https://pypi.org/project/keras-explain/
from keras_explain.lrp import LRP
from keras_explain.grad_cam import GradCam
from keras_explain.guided_bp import GuidedBP

Using TensorFlow backend.


# Model

In [2]:
model_path = "trained_models/C2D_AE_128_3x3_HAM10000/model.h5"

def get_model(model_path, rec = True, max_value=1000):
    if rec: model = C2D_AE_128_3x3(isTrain = True)
    else: model = C2D_AE_128_3x3(isTrain = False, max_value = max_value)
    model.model.load_weights(model_path)
    return model.model

# c2d_model_prb = get_model(False)
# c2d_model_rec = get_model(True)

# Data

In [3]:
DATASET = "HAM10000"
TEST_DIR = join_paths(["TEST_SETS/", DATASET, ""])

test_dict = dict()
for class_dir in read_directory_contents(TEST_DIR):
    test_dict[class_dir.split("/")[-1]] = np.array([read_image(file, resize_to=(128,128))/255. for file in read_directory_contents(class_dir)])
    
# 1 -> High Loss -> Anomaly
# 0 -> Low Loss -> Normal
X_test = list()
y_test = list()
for key in set(test_dict.keys()):
    X_test.append(test_dict[key])
    y_test += [0 if "normal" in key else 1]*len(test_dict[key])
X_test = np.concatenate(X_test)
y_test = np.array(y_test)

In [4]:
xai_results = dict()

In [5]:
de_methods = {
    "Gradient_Inputs": "grad*input",
    "Saliency": "saliency",
    "Integrated_Gradients": "intgrad",
    "DeepLIFT": "deeplift",
    "e-LRP": "elrp",
    "Occlusion": "occlusion"
}

In [ ]:
with DeepExplain(session=K.get_session()) as de:
    model = get_model(True)
    input_tensor = model.layers[0].input
    de_model = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    target_tensor = de_model(input_tensor)

    for method_name, method_tag in de_methods.items():
        print("-"*40)
        print("Method: %s"%method_name)
        print("-"*40)
        try: attributions = de.explain(method_tag, target_tensor, input_tensor, X_test, ys=X_test, batch_size = 64) # np.expand_dims(y_test, axis = -1))
        except Exception as e: 
            print("ERROR:", e)
            continue
        xai_results[method_name] = attributions

#     print("-"*40)
#     print("Method: %s"%method_name)
#     print("-"*40)
#     try: xai_results["Shapely"] = de.explain('shapley_sampling', target_tensor, input_tensor, X_test, ys=X_test, samples=2, sampling_dims=[1,2,3])
#     except Exception as e: print("ERROR:", e)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
----------------------------------------
Method: Gradient_Inputs
----------------------------------------



----------------------------------------
Method: Saliency
----------------------------------------
----------------------------------------
Method: Integrated_Gradients
----------------------------------------
----------------------------------------
Method: DeepLIFT
----------------------------------------
ERROR: 'NoneType' object cannot be interpreted as an integer
----------------------------------------
Method: e-LRP
----------------------------------------
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


----------------------------------------
Method: Occlusion
----------------------------------------


# LIME

In [ ]:
from _lime import *

In [ ]:
model = get_model(False)
le = LimeExplainer(model, num_features = 128*128, num_samples = 1000, batch_size = 64)
lime_results = le.explain(X_test, results_path = "", mask_features = 5, anomaly_type = None, save_result = False)
lime_results = np.array(lime_results)

xai_results["LIME"] = lime_results

# LRP, GuidedBP, GradCAM

# remove /tmp/checkpoint and all other keras files in /tmp/ if GuidedBP isnt working

In [ ]:
!rm /tmp/checkpoint 
!rm /tmp/gb_keras.h5 
!rm -rf /tmp/guided_backprop_ckpt.*

In [ ]:
model = get_model(True)

In [ ]:
ke_methods = {
    "GuidedBP": GuidedBP(model),
    "GradCAM": GradCam(model, layer = -2),
}

In [ ]:
k_explainer_results = dict([(key, list()) for key in list(ke_methods.keys())])

for method, explainer in ke_methods.items():
    for xs in X_test:
        exp = explainer.explain(xs, 1)
        k_explainer_results[method].append(normalize(exp[0]))
        
for key in k_explainer_results.keys():
    xai_results[key] = np.array(k_explainer_results[key])

# SHAP

In [ ]:
from _shap import *

In [ ]:
ir_train = HAM10000()
X_train = ir_train.data

In [ ]:
model = get_model(False)

In [ ]:
del se

In [ ]:
se = SHAP_Explainer(blend_alpha=0.45)
shap_out = se.explain(model, X_train[:1000], X_test/255.)
xai_results["SHAP"] = shap_out

# Save results

In [ ]:
print(xai_results.keys())
dump_pickle(xai_results, "results/xai_results_%s.pkl"%(DATASET))

In [ ]:
exit()

1. LRP
2. LIME
3. SHAP
4. COUNTER_FACTUALS
5. GUIDED_BP
6. GRAD_CAM
7. GRADIENT_INPUTS
8. INTEGRATED_GRADIENTS
9. SALIENCY
10. OCCLUSION